In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type = 'table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Overview of the Data

In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Above table shows the first five rows of facts table.
Here is more information about the individuals columns:


1. code - country code 
2. name - name of the country
3. area - total area (land+water) of the country in sq.km 
4. area_land - total land area in sq.km of the country
5. area_water - total water area in sq.km of the country
6. population - population of the country
7. population_growth - growth rate of population of a country expressed in percentage terms
8. birth_rate - birth rate of a country expressed as number of births per 1000
9. death_rate - death rate of a country expressed as number of births per 1000

# summary statistics

In [32]:
%%sql
SELECT MIN(population) AS min_population, 
       MAX(population) AS max_population,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
FROM facts

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


Facts table has an entry with population 0 and another with population close to 7.2 billion that is the world population
These are the outliers which need further investigation

# Investigating outliers

In [44]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population)
                   FROM facts) OR population == (SELECT MAX(population)
                                               FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Antarctica is the country with 0 population. This is true as Antarctica has no indigenous population except for research stations

The table also has an entry of the World with whole world populaiton

# Revisiting summary statistics

In [33]:
%%sql
SELECT MIN(population),  MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name <> 'World'

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


# Exploring avg population and area 

In [45]:
%%sql
SELECT AVG(population),AVG(area)
FROM facts
WHERE name <> 'World'

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


# Countries with high population density 

In [36]:
%%sql
SELECT name,population,area
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name <> 'World') AND area < (SELECT AVG(area)
                                                      FROM facts
                                                     WHERE name <> 'World')

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


# Country with the highest population

In [7]:
%%sql
SELECT *
FROM facts 
WHERE population == (SELECT MAX(population)
                   FROM facts where name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


# Country with highest population growth

In [48]:
%%sql
SELECT *
FROM facts 
WHERE population_growth == (SELECT MAX(population_growth)
                           FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


# Countries with high water to land area ratio

In [54]:
%%sql
SELECT name AS country,CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) AS water_land_ratio
FROM facts
ORDER BY water_land_ratio DESC
LIMIT 10

Done.


country,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


# Countries with water to land ratio greater than unity

In [38]:
%%sql
SELECT name AS country,area_water,area_land
FROM facts
WHERE area_water/area_land > 1

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


# Countries that will add most people to their population next year

In [58]:
%%sql 
SELECT name AS country,population, population_growth, population*population_growth AS people_added
FROM facts
WHERE name <> 'World'
ORDER BY people_added DESC
LIMIT 10

Done.


country,population,population_growth,people_added
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.20000005
Pakistan,199085847,1.46,290665336.62
Ethiopia,99465819,2.89,287456216.91
Bangladesh,168957745,1.6,270332392.0
United States,321368864,0.78,250667713.92000002
Indonesia,255993674,0.92,235514180.08
"Congo, Democratic Republic of the",79375136,2.45,194469083.20000002
Philippines,100998376,1.61,162607385.36


In [59]:
%%sql 
SELECT name,population, population_growth, population*population_growth AS people_added
FROM facts
WHERE population*population_growth = (SELECT MAX(population*population_growth)
                                     FROM facts
                                     WHERE name <> 'World')

Done.


name,population,population_growth,people_added
India,1251695584,1.22,1527068612.48


# Countries with death rate higher than birth rate

In [40]:
%%sql
SELECT name,birth_rate,death_rate
FROM facts
WHERE death_rate > birth_rate

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


# Countries with high population to area ratio

In [61]:
%%sql
SELECT name AS country ,population,area, CAST(population AS FLOAT)/CAST(area AS FLOAT) AS population_density 
FROM facts
WHERE name <> 'World'
ORDER BY population_density DESC
LIMIT 20

Done.


country,population,area,population_density
Macau,592731,28,21168.964285714286
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.279770444763
Hong Kong,7141106,1108,6445.041516245487
Gaza Strip,1869055,360,5191.819444444444
Gibraltar,29258,6,4876.333333333333
Bahrain,1346613,760,1771.8592105263158
Maldives,393253,298,1319.6409395973155
Malta,413965,316,1310.01582278481
Bermuda,70196,54,1299.9259259259259


In [41]:
%%sql
SELECT name AS country ,population,area, CAST(population AS FLOAT)/CAST(area AS FLOAT) AS population_density 
FROM facts
WHERE population/area = (SELECT MAX(population/area)
                        FROM facts
                        WHERE name <> 'World')

Done.


name,population,area
Macau,592731,28
